## blackjack sim

##### Gameplay:
1. Place bet
2. Deal cards
4. View hand and select action (player_action)
5. Compute dealer's outcome, determine winner and payout (finish_round)


In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
import copy #can't believe this is necessary...

In [2]:
card_deck = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

def compute_value(hand):
    val=0
    softhard='hard'
    for card in hand:
        try: 
            val += int(card)
        except: 
            if card=='A':
                val+=11
                softhard='soft'
                if val > 21:
                    val-=10
                    softhard='hard'
            else: 
                val+=10

    return val,softhard

def hand_to_string(hand):
    # Generate pretty string of hand and its value
    value, softhard = compute_value(hand)
    if softhard=='soft':
        return softhard +' '+str(value)+' '+str(hand)
    else: 
        return str(value)+' '+str(hand)

def splitting_is_allowed(hand):
    # check for legal splitting - two cards of same value in hand
    allowed = True
    if len(hand) > 2:
        allowed = False
    if compute_value([hand[0]])[0] != compute_value([hand[1]])[0]:
        allowed = False
    return allowed

def doubling_is_allowed(hand):
    # check for legal doubling - there are only two cards in hand
    if len(hand)==2:
        return True
    else:
        return False

def blackjack(hand):
    # Check for blackjack
    return compute_value(hand)[0]==21 and len(hand)==2
        
def draw_from_deck():
    # Get a random card from the deck
    return card_deck[random.randint(0,12)]

In [43]:
class Game:
    def __init__(self):               
        # Set game rules here
        self.hit_on_soft_17 = True
        self.blackjack_payout = 6/5
        
        
    #Three lists: hands, bets, payouts. Int num_payouts. 
    #Lists change length so keep track of what's been finished with int self.num_payouts.
    #
        self.player_hands=[]
        self.dealer_hand = []
        self.bets=[]
        self.payouts=[]

    def play_round_interactive(self, bet):
        total_payout = 0
        self.hands_completed = 0
        self.bet = bet
        self.deal()
        
        # play game for initial hand
        hand_number = 0
        if blackjack(self.player_hands[hand_number]):
            self.flip_hidden_card()
            print('Dealer has: '+hand_to_string(self.dealer_hand))
            payout = self.finish_round(hand_number)
            total_payout+=payout
        else:
            action = self.player_action_interactive(hand_number)
            if action =='split':
                print('hands completed: '+str(self.hands_completed))
                print('len player hands: '+str(len(self.player_hands)))
                while self.hands_completed < len(self.player_hands):
                    if blackjack(self.player_hands[hand_number]):
                        if dealer_hidden_card !=0: 
                            self.flip_hidden_card()
                        print('Dealer has: '+hand_to_string(self.dealer_hand))
                        payout = self.finish_round(hand_number)
                        total_payout+=payout
                    else: 
                        print('action on hand number '+str(hand_number))
                        self.player_action_interactive(hand_number)
                        self.dealer_action()
                        payout = self.finish_round(hand_number)
                        total_payout+=payout
                    hand_number+=1
                    print('hands completed: '+str(self.hands_completed))
                    print('len player hands: '+str(len(self.player_hands)))

            else: 
                self.dealer_action()
                payout = self.finish_round(hand_number)
                total_payout+=payout
        return total_payout #dict with hand, bet, payout??
            
    def deal(self):
        #deal initial hand for player and dealer
        self.dealer_hand = [draw_from_deck()]
        self.dealer_hidden_card = draw_from_deck()
        self.player_hands = [[draw_from_deck(),draw_from_deck()]]
        self.bets = [self.bet]
        
        print('Player has: '+hand_to_string(self.player_hands[0]))
        print('Dealer has: '+hand_to_string(self.dealer_hand))

    def player_action_interactive(self,hand_number):
        player_hand = self.player_hands[hand_number]
        while compute_value(player_hand)[0]<22:
            print('stand/hit/double/split?')
            do = input() # prompt user for action
            if do == 'stand':
                break
            elif do == 'hit':
                player_hand.append(draw_from_deck())
                self.player_hands[hand_number] = player_hand
                print('Player has: '+hand_to_string(player_hand))
            elif do == 'split':
                if splitting_is_allowed(player_hand):
                    new_hands = self.split_hands(player_hand)
                    print(new_hands)
                    del self.player_hands[hand_number]
                    game.player_hands.append(new_hands[0])
                    game.player_hands.append(new_hands[1])
                    break
                else: 
                    print('invalid choice')
            elif do == 'double':
                if doubling_is_allowed(player_hand):
                    self.bet = self.bet*2
                    player_hand.append(draw_from_deck())
                    self.player_hands[hand_number] = player_hand
                    print('Player has: '+hand_to_string(player_hand))
                    break
                else:
                    print('invalid choice')
            else:
                print('invalid choice')
        
        return do
        
    def dealer_action(self):
        if self.dealer_hidden_card != 0:
            self.flip_hidden_card()
        while compute_value(self.dealer_hand)[0]<18:
            self.dealer_hand.append(draw_from_deck())
        if compute_value(self.dealer_hand)[0]==17 and compute_value(self.dealer_hand)[1]=='soft':
            if self.hit_on_soft_17: 
                self.dealer_hand.append(draw_from_deck())            
        print('Dealer has: '+hand_to_string(self.dealer_hand))
        
    def finish_round(self,hand_number):
        # compute payout based upon player and dealer hands
        player_hand = self.player_hands[hand_number]
        dealer_hand = self.dealer_hand
        bet = copy.deepcopy(self.bet)
        if blackjack(player_hand):
            if blackjack(dealer_hand):
                payout=0
                self.hands_completed+=1
                print('Blackjack, push, payout $0')
                return payout
            else:
                payout=self.bet*self.blackjack_payout
                self.hands_completed+=1
                print('Blackjack, payout $'+str(payout))
                return payout
        if compute_value(player_hand)[0]>21:
            payout = -bet
            self.hands_completed+=1
            print('Player bust, payout $'+str(payout))
            return payout
        if compute_value(dealer_hand)[0]>21:
            payout = bet
            self.hands_completed+=1
            print('Dealer bust, payout $'+str(payout))
            return payout
        if compute_value(player_hand)[0] > compute_value(dealer_hand)[0]:
            payout = bet
            self.hands_completed+=1
            print('Player wins, payout $'+str(payout))
            return payout
        elif compute_value(player_hand)[0] == compute_value(dealer_hand)[0]:
            payout = 0
            print('Push, payout $'+str(payout))
            self.hands_completed+=1
            return payout
        else: 
            payout = -bet
            print('Dealer wins, payout $'+str(payout))
            return payout
        
    def flip_hidden_card(self):
        self.dealer_hand.append(self.dealer_hidden_card)
        self.dealer_hidden_card = 0
    
    def split_hands(self,hand):
        # create new split hands
        return [[hand[0],draw_from_deck()],[hand[1],draw_from_deck()]]


In [46]:
game = Game()
i=1
while i<10:
    print('\nNew Round!')
    game.play_round_interactive(10)
    i+=1


New Round!
Player has: soft 14 ['A', '3']
Dealer has: 10 ['Q']
stand/hit/double/split?
hit
Player has: soft 24 ['A', '3', 'K']
Dealer has: soft 21 ['Q', 'A']
Player bust, payout $-10

New Round!
Player has: 19 ['9', 'K']
Dealer has: 10 ['Q']
stand/hit/double/split?
stand
Dealer has: 20 ['Q', 'Q']
Dealer wins, payout $-10

New Round!
Player has: 14 ['6', '8']
Dealer has: 10 ['Q']
stand/hit/double/split?
hit
Player has: 16 ['6', '8', '2']
stand/hit/double/split?
hit
Player has: 26 ['6', '8', '2', 'Q']
Dealer has: 24 ['Q', '5', '9']
Player bust, payout $-10

New Round!
Player has: 7 ['5', '2']
Dealer has: 6 ['6']
stand/hit/double/split?
hit
Player has: 16 ['5', '2', '9']
stand/hit/double/split?
stand
Dealer has: 22 ['6', 'J', '6']
Dealer bust, payout $10

New Round!
Player has: 15 ['J', '5']
Dealer has: 6 ['6']
stand/hit/double/split?
hit
Player has: 20 ['J', '5', '5']
stand/hit/double/split?
stand
Dealer has: 24 ['6', '3', '7', '8']
Dealer bust, payout $10

New Round!
Player has: 15 ['9

KeyboardInterrupt: Interrupted by user